In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import seaborn as sns
import quandl
from scipy.optimize import minimize
from pandas import Series
from pandas import DataFrame

In [47]:
df = pd.read_csv('Exam_Industry.csv',index_col=0,parse_dates=True)
mk = pd.read_csv('Exam_Market.csv',index_col=0,parse_dates=True)

In [48]:
stock_names 

Index(['Cnsmr', 'Manuf', 'HiTec', 'Hlth ', 'Other', 'downside_returns'], dtype='object')

In [49]:
stock_names  = df.columns

In [50]:
sharpe_ratio = []
treynor_ratio = []
sortino_ratio =[]
jensens_alpha =[]
information_ratio=[]

In [51]:
## sharp ratio
for i in stock_names:
    ex_return = df[i]
    mean_ex_return = ex_return.mean()
    std__ex_return = ex_return.std()
    sharpe_r = mean_ex_return/std__ex_return
    sharpe_ratio.append(sharpe_r)

#### Before we calculate treynor ratio, we have to get beta again

In [52]:
import statsmodels.api as sm
mk_excess_return = mk['Mkt']
mk_add_constant = sm.add_constant(mk_excess_return)
stock_names  = df.columns

result_pd = []
for stock in stock_names: 
    model = sm.OLS(endog = df[stock], exog = mk_add_constant) 
    result = model.fit() 
    result_p = result.params
    result_pd.append(result_p)

resultp = pd.DataFrame(result_pd)
resultp = resultp.rename(columns = {'const': 'alpha', 'Mkt': 'Beta'}, inplace = False)
Name = ['Cnsmr', 'Manuf', 'HiTec', 'Hlth ', 'Other']
Name_table = pd.DataFrame(data = Name,columns = ['Company'])
table=pd.concat([Name_table,resultp],axis=1)
table = table.set_index('Company')


In [53]:
table

,alpha,Beta
Company,,
Cnsmr,0.368663,0.786378
Manuf,-0.237342,0.995841
HiTec,0.240676,0.995339
Hlth,0.318161,0.741340
Other,-0.309610,1.268648


In [54]:
table2 = table['Beta'].T
table2

Company
Cnsmr    0.786378
Manuf    0.995841
HiTec    0.995339
Hlth     0.741340
Other    1.268648
Name: Beta, dtype: float64

In [55]:
## treynor ratio
for i in stock_names:
    ex_return = df[i]
    mean_ex_return = ex_return.mean()
    treynor_r = mean_ex_return/table2[i]
    treynor_ratio.append(treynor_r)

In [56]:
 treynor_ratio

[1.7089160036058637,
 1.0017708593836943,
 1.4819071412296514,
 1.6692750665054858,
 0.9960569200895226]

In [57]:
## jensen's alpha
for i in stock_names:
    ex_return = df[i]
    mean_ex_return = ex_return.mean()
    ex_mk_return = mk['Mkt']
    mean_ex_mk_return = ex_mk_return.mean()
    jensens_a = mean_ex_return -(table2[i]*mean_ex_mk_return)
    jensens_alpha.append(jensens_a)

In [58]:
jensens_alpha

[0.3686633743987672,
 -0.23734200109116343,
 0.24067593546006827,
 0.31816145775324967,
 -0.30961010365347796]

In [60]:
mk.mean()

Mkt    1.240104
dtype: float64

In [62]:
## sortino_ratio
target = 1.24
for i in stock_names:
    df['downside_returns'] = 0
    df.loc[df[i] < target, 'downside_returns'] = df[i]**2
    ex_return = df[i]
    mean_ex_return = ex_return.mean()
    down_stdev = np.sqrt(df['downside_returns'].mean())
    sortino_r = (mean_ex_return)/down_stdev
    sortino_ratio.append(sortino_r )

In [63]:
sortino_ratio

[0.7255174267091338,
 0.35983427762320175,
 0.6348399760030492,
 0.5079420321059436,
 0.3530238543274675]

In [64]:
## information ratio
for i in stock_names:
    ex_return = df[i]-1.24
    mean_ex_return = ex_return.mean()
    std__ex_return = ex_return.std()
    information_r = mean_ex_return/std__ex_return
    information_ratio.append(information_r )

In [65]:
information_ratio

[0.029359473812757927,
 -0.05417889433146324,
 0.05288047574366025,
 -0.0006254905852370766,
 0.0042240770995652785]

In [66]:
## Sorting data
sharpe_ratio = pd.DataFrame(sharpe_ratio)
sharpe_ratio = sharpe_ratio.rename(columns = {0: 'sharpe_ratio'}, inplace = False)
sortino_ratio = pd.DataFrame(sortino_ratio)
sortino_ratio = sortino_ratio.rename(columns = {0: 'sortino_ratio'}, inplace = False)
treynor_ratio = pd.DataFrame(treynor_ratio)
treynor_ratio = treynor_ratio.rename(columns = {0: 'treynor_ratio'}, inplace = False)
jensens_alpha = pd.DataFrame(jensens_alpha)
jensens_alpha= jensens_alpha.rename(columns = {0: 'jensens_alpha'}, inplace = False)
information_ratio= pd.DataFrame(information_ratio)
information_ratio= information_ratio.rename(columns = {0: 'information_ratio'}, inplace = False)

In [68]:
## Create a table showing the performance metrics for the ten industry portfolios
Name = ['Cnsmr', 'Manuf', 'HiTec', 'Hlth ', 'Other']
Name_table = pd.DataFrame(data = Name,columns = ['Company'])
table_sum=pd.concat([Name_table,sharpe_ratio, sortino_ratio,treynor_ratio,jensens_alpha,information_ratio],axis=1)
table_sum = table_sum.set_index('Company')

In [69]:
table_sum

,sharpe_ratio,sortino_ratio,treynor_ratio,jensens_alpha,information_ratio
Company,,,,,
Cnsmr,0.379906,0.725517,1.708916,0.368663,0.029359
Manuf,0.222979,0.359834,1.001771,-0.237342,-0.054179
HiTec,0.331909,0.634840,1.481907,0.240676,0.052880
Hlth,0.309618,0.507942,1.669275,0.318161,-0.000625
Other,0.225737,0.353024,0.996057,-0.309610,0.004224
